In [ ]:
image_path = '/content/Assignment-main/Damage Identification/images'
json_path = '/content/Assignment-main/Damage Identification/data'

In [ ]:
import os
import json
import numpy as np
from shapely.geometry import Polygon

In [ ]:
for i in range(len(os.listdir(image_path))):
  subpath1 = str(i+1) + ".jpg"
  subpath2 = str(i+1) + "-car-parts.json"
  subpath3 = str(i+1) + "-damages.json"
  img_path = os.path.join(image_path,subpath1)
  data_path = os.path.join(json_path,subpath2)
  damage_path = os.path.join(json_path,subpath3)
  output = solve(img_path,data_path,damage_path)
  for i in output:
    print(i)
  print()

Found 2 Dent&Scratch(zoom) on part Bumper with percentage 4.908251709342389

Found 1 Dent&Scratch on part Bumper with percentage 1.6164428416402563
Found 1 Dent&Scratch on part Light with percentage 2.4089991014091106
Found 1 Dent&Scratch on part Bonnet with percentage 0.3652208958639137
Found 2 Dent&Scratch on part Fender with percentage 33.84295869041098
Found 1 Dent&Scratch on part Door with percentage 27.025451413905035

Found 3 Dent&Scratch on part Bumper with percentage 2.5321780561542644

Found 1 Dent&Scratch on part Light with percentage 1.7022935075758017
Found 1 Dent&Scratch on part Door with percentage 44.30896955635947
Found 1 Dent&Scratch on part RockerPanel with percentage 1.196950985316282
Found 1 Dent&Scratch on part Fender with percentage 99.98886789412782

Found 1 Broken on part Bonnet with percentage 0.851846009179435
Found 1 Broken on part Windshield with percentage 98.9786505398943
Found 1 Broken on part Roof with percentage 0.09131893932338693



In [ ]:
def get_denormalized_cordinates(data_path):
  denormalized_points = []
  data = open(data_path)
  info = json.load(data)
  for i in info:
    if(i['type']=="polygonlabels"):
      points = i['value']['points']
      width = i['original_width']
      height = i['original_height']
      for j in range(np.shape(points)[0]):
        points[j][0] = points[j][0]*width/100
        points[j][1] = points[j][1]*height/100
      carpart = i['value']['polygonlabels']
      temp = {}
      points = np.asarray(points,np.int32)
      temp['id'] = i['id']
      temp['part'] = carpart
      temp['points'] = points
      denormalized_points.append(temp)
  data.close()
  return denormalized_points

In [ ]:
def solve(img_path,data_path,damage_path):
  a = []
  carparts = get_denormalized_cordinates(data_path)
  damages = get_denormalized_cordinates(damage_path)
  for i in carparts:
    for j in damages:
      p1 = Polygon(i['points'])
      p2 = Polygon(j['points'])
      if (p1.intersects(p2)==True):
        dictionary = {}
        dictionary['part-id'] = i['id']
        dictionary['part'] = i['part']
        dictionary['damages'] = j['part']
        dictionary['carpoly'] = i['points']
        dictionary['damagepoly'] = j['points']
        a.append(dictionary)
  ans = {}
  for i in a:
    if i['part-id'] in ans:
      p2 = i['damagepoly']
      p2 = Polygon(p2)
      temp['damage'] = temp['damage'] + p1.intersection(p2).area
      temp['count'] = temp['count'] + 1
    else:
      temp = {}
      temp['name'] = i['part']
      temp['damages'] = j['part']
      p1 = i['carpoly']
      p2 = i['damagepoly']
      p1 = Polygon(p1)
      p2 = Polygon(p2) 
      temp['count'] = 1
      temp['area'] = p1.area
      temp['damage'] = p1.intersection(p2).area
      ans[i['part-id']] = temp
  # print(ans)
  string = []
  for i in ans:
    temp = "Found " + str(ans[i]['count']) + " " + ans[i]['damages'][0] + " on part " + ans[i]['name'][0] + " with percentage " + str(ans[i]['damage']*100/ans[i]['area'])
    string.append(temp)
  return string